# TEP detailed analysis

In [ ]:
%matplotlib inline
%autosave 5

In [ ]:
# necessary modules 
import pyfesom2 as pf
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.patches as mpatches
from shapely.geometry.polygon import LinearRing

from netCDF4 import Dataset

import time
from datetime import date


import cartopy.crs as ccrs
import cartopy.feature as cfeature
#import pangaeapy.pandataset as pd

import warnings
warnings.simplefilter("ignore")

# -------------------------------------------------------------------------------
# own py_recom modules 
#!jupyter nbconvert --to=python plot_Taylor_normalized.ipynb

#from load_interp_WOA_python3 import WOAdata
from plot_Taylor_normalized import plt_Taylor_norm
from Chlsurf_OCCCI_Comparison_python3 import Chlsurf_OCCCI_comp


In [ ]:
# run specification 
runid      =  'fesom'
layerwise    = False
resultpath = '/work/ollie/mozeisin/results/f2r1.2/tep1' 
meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'

# period of analysis 
first_year = 1950
last_year  = 1980

# -------------------------------------------------------------------------------
# specification of analysis 
mapproj = 'pc'
#Map projection. Options are Mercator (merc), Plate Carree (pc),
#North Polar Stereo (np), South Polar Stereo (sp),  Robinson (rob)
#Robinson projection is quite time consuming.

# time arrays
years = np.arange(first_year,last_year+1,1)
years_last10 = np.arange(last_year-10,last_year+1,1)
months = np.arange(1,13,1)

# -------------------------------------------------------------------------------
# observation data 
# OCCCI: CHl.a 
matfileChlsurf           = '/work/ollie/projects/MarESys/evaluation/climatology_annual_chl_1deg_OCCCI_2012_2015.mat'

# -------------------------------------------------------------------------------
# export of analysis 
today = date.today().strftime("_%Y_%m_%d")

savepath = '/home/ollie/mozeisin/evaluation/mo_files/tep1/'
savefig = True

htmlname     =  'tep_analysis' + today +'.html'
htmlpath = '/home/ollie/mozeisin/evaluation/mo_files/html/'
    

In [ ]:
# just to check
print('Processing years {4} to {5}\n\nReading out of {0}\nStoring graphs to {1}\nStoring html to {2} as {3}'.format(
    resultpath, savepath, htmlpath, htmlname,years[0],years[-1]))

print('\nLast ten years are {0}'.format(years_last10))

# Load mesh

In [ ]:
### mesh initialization 

mesh = pf.load_mesh(meshpath)

lon_fesom = mesh.x2
lat_fesom = mesh.y2

#print(mesh.zlev)

In [ ]:
# -------------------------------------------------------------------------------
### load FESOM diag 
meshdiag = resultpath+'/fesom.mesh.diag.nc'
diag = pf.get_meshdiag(mesh,meshdiag=meshdiag) 

# level depth  
# c.f. Danilov et al. (2017)
# "in the vertical direction, the horizontal velocities and scalars are located at mid-levels" 
layer_depths = -diag['Z'].values # depth of layers
layer_depths = layer_depths[:-1] # lowest depth seems to be benthic?!

# --> layer thickness
delta_z = np.abs(np.diff(mesh.zlev))

# control:
if True:
    print('Lenght of diag_Z (layer): {1} with values: \n{0}\n\nLenght of mesh.zlev: {3} with values: \n{2} \n\nLenght of delta_z (layer): {4} with values:\n{5}'.format(
        diag['Z'].values, len(diag['Z'].values), mesh.zlev, len(mesh.zlev),len(delta_z),delta_z))

In [ ]:
# -------------------------------------------------------------------------------
### load nodal area 

nod_area = diag.rename_dims({"nl": "nz1", "nod_n": "nod2"}).nod_area
nod_area.load()
nod_area = np.swapaxes(np.array(nod_area.data),0,1)

# -------------------------------------------------------------------------------
### nodal volume
delta_aux = np.tile(delta_z,(126858,1))#mesh.nod2))
#print(np.shape(delta_z),np.shape(delta_aux))

nod_vol = nod_area[:,:-1] * delta_aux
print('Shape of nod_vol: {0}'.format(np.shape(nod_vol)))

## test
if False:
    for idepth in range(0,40,5):
        vol_manual = nod_area.data[1000,idepth] * delta_aux[1000,idepth]
        print('***\ndepth = {4}\narea = {0}, delta = {1}, vol = {2}\nControl: {5} --> nod_vol = {3}\n***'.format(
            nod_area.data[1000,idepth], delta_aux[1000,idepth],vol_manual,
            nod_vol[1000,idepth],mesh.zlev[idepth],nod_vol[1000,idepth]==vol_manual))
        

# Own grids for interpolation

In [ ]:
# -------------------------------------------------------------------------------
### grid for interpolation to 1x1 

lat         = np.arange(-89.5,90.,1.)
lon         = np.arange(-179.5,180.,1.)
latdic, londic = np.meshgrid(lat, lon)


# -------------------------------------------------------------------------------
### grid for interpolation to 1x1 

lat05         = np.arange(-89.5,90.,.5)
lon05         = np.arange(-179.5,180.,.5)
latdic05, londic05 = np.meshgrid(lat05, lon05)

# Define regions

In [ ]:
### Arctic Oean, N of 70 
box_AO =[-180, 180, 70, 90] 
left, right, down, up = box_AO
mask_AO = ((mesh.x2 >= left) & 
           (mesh.x2 <= right) & 
           (mesh.y2 >= down) & 
           (mesh.y2 <= up))

mask_AO_1x1_lon = ((lon >= left) & (lon <= right))
mask_AO_1x1_lat = ((lat >= down) & (lat <= up))

#nod_area_AO = nod_area[:,mask_AO]
#nod_area_AO_surface = nod_area[0,mask_AO]

# -------------------------------------------------------------------------------
### Fram Strait 
box_FRAM =[-20, 12, 77, 82]
left, right, down, up = box_FRAM
mask_FRAM = ((mesh.x2 >= left) & 
           (mesh.x2 <= right) & 
           (mesh.y2 >= down) & 
           (mesh.y2 <= up))

mask_FRAM_1x1_lon = ((lon >= left) & (lon <= right))
mask_FRAM_1x1_lat = ((lat >= down) & (lat <= up))

# highlight Fram Strait with red box
ring1 = LinearRing(list(zip([-20,-20,12,12], [77,82,82,77])))

# -------------------------------------------------------------------------------
### Souther Oean 
box_SO =[-180, 180, -90, -50] 
left, right, down, up = box_SO
mask_SO = ((mesh.x2 >= left) & 
           (mesh.x2 <= right) & 
           (mesh.y2 >= down) & 
           (mesh.y2 <= up))

#nod_area_SO = nod_area[:,mask_SO]
#nod_area_SO_surface = nod_area[0,mask_SO]
#print(np.shape(nod_area_SO_surface))
        

# Functions

In [ ]:
### define standard grid layout for plotting 
def mygrid(m,grid=False):
    '''
    Draw cartopy natural earth feature to axis m
    '''
    #m.coastlines(resolution='110m', color='black', linewidth=1)
    land_poly = cfeature.NaturalEarthFeature('physical', 'land', '110m',edgecolor='k',facecolor=cfeature.COLORS['land'])
    m.add_feature(land_poly)

    if grid:
        g1 = m.gridlines(draw_labels = True)
        g1.xlabels_top = False
        g1.ylabels_right = False
        g1.xlabel_style = {'size': 16, 'color': 'gray'}
        g1.ylabel_style = {'size': 16, 'color': 'gray'}
        

# -------------------------------------------------------------------------------
### omit low Chl values, omit same TEP datapoints 
def set_Chl_min(value,Chl_interp,TEP_interp):
    ind = np.where(Chl_interp > value)
    Chl_interp_min = Chl_interp[ind]
    print('Number of Chl.a datapoints omitted: {0}'.format(len(Chl_interp) - len(Chl_interp_min) ))

    # apply to TEP as well
    TEP_interp_min = TEP_interp[ind]
    
    return [Chl_interp_min,TEP_interp_min]


# -------------------------------------------------------------------------------
### linear regression 
def get_res(x,y):
    res = stats.linregress(x,y)
    log = 'Regression: \nslope = %.2f intercept = %.2f, std error = %.3f\nR-squared = %.5f' % (
        res.slope, res.intercept, res.stderr, res.rvalue**2)
    print(log)
    return res, log


# -------------------------------------------------------------------------------
### get_monthly_data 
def get_monthly_data(var_name, years=years_last10, verbose=True, only2D = False): 
    months = np.arange(1,13,1)
    
    if only2D:
        var_monthly_aux = np.zeros((len(years),len(months),len(mesh.x2)))
    else:
        var_monthly_aux = np.zeros((len(years),len(months),len(mesh.x2),len(mesh.zlev)-1))

    if verbose:
        print('Processing all depths vor variable "{}"'.format(var_name))

    months = np.arange(1,13,1)
    
    # load data for one depth and calculate monthly mean
    for i in range(len(years)):
        ind_year = int(years[i])

        if verbose:
            print('loading {0}'.format(ind_year))

        var_raw = pf.get_data(resultpath, var_name, ind_year, mesh, how="ori", 
                            #depth=depth, 
                            compute=False,  silent=True)#np.invert(verbose))

        for m in range(len(months)):
            if only2D:
                var_monthly_aux[i,m,:] = var_raw[m,:]
            else:
                var_monthly_aux[i,m,:,:] = var_raw[m,:,:]

    if verbose:
        print('Processed {0} years of input data\nResulting shape: {1}'.format(
            len(years_last10),np.shape(var_monthly_aux)))

    # monthly average over years
    var_monthly_alldepths = np.nanmean(var_monthly_aux,axis=0)
    print('\nMonthly mean over years \nResulting shape {0}'.format(np.shape(var_monthly_alldepths)))

    return var_monthly_alldepths

In [ ]:
class var_monthly:   
    '''
    class var_monthly()
    '''
    def __init__(self,resultpath, var_name, years, mesh, silent=False, only2D=False):
        
        import numpy as np
        import pyfesom2 as pf
        import matplotlib.pyplot as plt
        #from netCDF4 import Dataset
        #import cartopy.crs as ccrs
    
        self.resultpath = resultpath
        self.var_name = var_name
        self.years = years
        self.mesh = mesh
        self.silent = silent
        self.only2D = only2D

        self.months = np.arange(1,13,1)
        
        if only2D:
            var_monthly_aux = np.zeros((len(self.years),len(self.months),len(self.mesh.x2)))
        else:
            var_monthly_aux = np.zeros((len(self.years),len(self.months),len(self.mesh.x2),len(self.mesh.zlev)-1))

        if not self.silent:
            print('Processing all depths vor variable "{}"'.format(self.var_name))

        # load data for one depth and calculate monthly mean
        for i in range(len(self.years)):
            ind_year = int(self.years[i])

            if not silent:
                print('loading {0}'.format(ind_year))

            var_raw = pf.get_data(self.resultpath, self.var_name, ind_year, self.mesh, 
                                  how="ori", compute=False, silent=True)

            for m in range(len(self.months)):
                if only2D:
                    var_monthly_aux[i,m,:] = var_raw[m,:]
                else:
                    var_monthly_aux[i,m,:,:] = var_raw[m,:,:]

        if not silent:
            print('Processed {0} years of input data\nResulting shape: {1}'.format(
                len(self.years),np.shape(var_monthly_aux)))

        self.data_monthly = var_monthly_aux


    def seasonalcycle(self):
        '''
        Calculate monthly average over years, i.e. seasonal cycle.
        Caculcated per depth level.
        '''
        # 
        mean_monthly_alldepths = np.nanmean(self.data_monthly,axis=0)
        print('\nSeasonal cycle over years {1} to {2} \nResulting shape {0}'.format(
            np.shape(mean_monthly_alldepths), self.years[0], self.years[-1]))
        
        self.data_seasonalcycle = mean_monthly_alldepths
        
        
    
    def volwmean(self, use_seasonal = True, depthrange=[0,-100]):
        '''
        Volume-weighted mean over specified depth range.
        '''
        
        self.use_seasonal = use_seasonal
        self.depthrange = depthrange
        
        ### retrieving depth indices
        i_depth0 = pf.ind_for_depth(self.depthrange[0], self.mesh)
        i_depth1 = pf.ind_for_depth(self.depthrange[1], self.mesh)
        
        if not self.silent:
            print('\nVolume-weighted mean:\nUsing depth range of {0} to {1} m, i.e. indices {2} to {3}'.format(
                self.depthrange[0], self.depthrange[1], i_depth0, i_depth1))
            
        ### load FESOM diag 
        meshdiag = self.resultpath+'/fesom.mesh.diag.nc'
        diag = pf.get_meshdiag(self.mesh,meshdiag=meshdiag) 

        ### layer depth  
        # c.f. Danilov et al. (2017)
        # "in the vertical direction, the horizontal velocities and scalars are located at mid-levels" 
        layer_depths = -diag['Z'].values # depth of layers
        layer_depths = layer_depths[:-1] # lowest depth seems to be benthic?!

        # --> layer thickness
        delta_z = np.abs(np.diff(self.mesh.zlev))
        delta_aux = np.tile(delta_z,(126858,1))
        
        ### load nodal area 
        nod_area = diag.rename_dims({"nl": "nz1", "nod_n": "nod2"}).nod_area
        nod_area.load()
        nod_area = np.swapaxes(np.array(nod_area.data),0,1)

        ### nodal volume
        nod_vol = nod_area[:,:-1] * delta_aux            
         
        ### processing data
        if self.use_seasonal:
            data = self.data_seasonalcycle
            nod_vol = np.tile(nod_vol,(12,1,1))
            
        else:
            data = self.data_monthly
            #nod_vol = np.tile(nod_vol, ())
            print('to be implemented')
        
        ### applying depth range
        data = np.squeeze(data[:,:,i_depth0:i_depth1])
        nod_vol = np.squeeze(nod_vol[:,:,i_depth0:i_depth1])
        
        if not self.silent:
            print('Shape of data: {0}'.format(np.shape(data)))
            print('Shape of nod_vol: {0}'.format(np.shape(nod_vol)))
        
        aux_w_data = np.nansum(data * nod_vol, axis = 2)
        aux_tot_vol = np.nansum(nod_vol, axis = 2) 
            
        self.data_monthly_wm =  aux_w_data / aux_tot_vol
        self.nod_vol = nod_vol   
        
        if not self.silent:
            print('Shape of data_monthly_wm: {0}'.format(np.shape(self.data_monthly_wm)))
        

# Read data

## TEP

In [ ]:
NCfesom = resultpath + '/TEP.fesom.1948.nc'
!ncdump -h $NCfesom

TEP_units = 'TEP-C [mmol m$^{-3}$]'


# get all-depths TEP data
TEP_alldepths = pf.get_data(resultpath, "TEP", years_last10, mesh, how="mean", runid='fesom')
    
print('Loaded TEP data with shape {0}'.format(np.shape(TEP_alldepths)))

In [ ]:
# get monthly, all-depths TEP data
TEP_monthly = var_monthly(resultpath, 'TEP', years_last10, mesh)

TEP_monthly.seasonalcycle()

TEP_monthly.volwmean()

In [ ]:
# get monthly, all-depths TEP data
#TEP_monthly_alldepths = get_monthly_data('TEP', years=years_last10, verbose=True)

TEP_seasonal_alldepths = TEP_monthly.data_seasonalcycle

print(np.shape(TEP_seasonal_alldepths))

## PCHO

In [ ]:
NCfesom = resultpath + '/PCHO.fesom.1948.nc'
#!ncdump -h $NCfesom

PCHO_units = 'org C [mmol m$^{-3}$]'


In [ ]:
# get PCHO data (all months)
#PCHO = pf.get_data(resultpath, "PCHO", years_last10, mesh=mesh, runid=runid, how="mean")#, depth=depth)

#PCHO_monthly_alldepths = get_monthly_data('PCHO', years=years_last10, verbose=True)

PCHO_monthly = var_monthly(resultpath, 'PCHO', years_last10, mesh)
PCHO_monthly.seasonalcycle()

PCHO_seasonal_alldepths = PCHO_monthly.data_seasonalcycle

## Chlorophyll

In [ ]:
### get CHLOROPHYLL data for all-months-mean

#PhyChl_Fram = pf.get_data(resultpath, "PhyChl", years_last10, mesh=mesh, runid=runid, how="mean")#, depth=depth)
#DiaChl_Fram = pf.get_data(resultpath, "PhyChl", years_last10, mesh=mesh, runid=runid, how="mean")#, depth=depth)
#Chl_SUM_Fram = PhyChl_Fram + DiaChl_Fram

#DiaChl_monthly_alldepths = get_monthly_data('DiaChl', years=years_last10, verbose=True)
#PhyChl_monthly_alldepths = get_monthly_data('PhyChl', years=years_last10, verbose=True)
#Chl_monthly_alldepths = DiaChl_monthly_alldepths + PhyChl_monthly_alldepths

PhyChl_monthly = var_monthly(resultpath, 'PhyChl', years_last10, mesh)
PhyChl_monthly.seasonalcycle()

DiaChl_monthly = var_monthly(resultpath, 'DiaChl', years_last10, mesh)
DiaChl_monthly.seasonalcycle()

Chl_seasonal_alldepths = DiaChl_monthly.data_seasonalcycle + PhyChl_monthly.data_seasonalcycle

# Analysis

## TEP

### Interpolating to 1x1 and converting to [$\mu$g L$^{-1}$]

In [ ]:
# interpolating and converting Arctic TEP [mug /L]    

conv = 12.01 # [mmol C/m3] -> [mug /L]  

# surface layer
TEP_monthly_0 = TEP_seasonal_alldepths[:,:,0]

if False: # iternp to 1x1
    # July
    TEP_monthly_0_interp_JUL = pf.fesom2regular(data = TEP_monthly_0[6,:],mesh = mesh,lons = londic,lats = latdic)
    TEP_monthly_0_interp_JUL = TEP_monthly_0_interp_JUL * conv

    # September
    TEP_monthly_0_interp_SEP = pf.fesom2regular(data = TEP_monthly_0[8,:],mesh = mesh,lons = londic,lats = latdic)
    TEP_monthly_0_interp_SEP = TEP_monthly_0_interp_SEP * conv

else: # interp to 0.5x0.5
    print('Interpolating to 0.5x0.5')
    
    # July
    TEP_monthly_0_interp_JUL = pf.fesom2regular(data = TEP_monthly_0[6,:],mesh = mesh,
                                                lons = londic05,lats = latdic05)
    TEP_monthly_0_interp_JUL = TEP_monthly_0_interp_JUL * conv

    # September
    TEP_monthly_0_interp_SEP = pf.fesom2regular(data = TEP_monthly_0[8,:],mesh = mesh,
                                                lons = londic05,lats = latdic05)
    TEP_monthly_0_interp_SEP = TEP_monthly_0_interp_SEP * conv





### Plotting 1x1 with NPS for July and September

In [ ]:
# North Polar Stereo Projection
fig = plt.figure(figsize=(15,10))
plot_extent = [-180, 180, 70, 90]

ax1 = plt.subplot(1,2,1,projection = ccrs.NorthPolarStereo())
m1 = ax1.pcolormesh(londic05, latdic05, TEP_monthly_0_interp_JUL, 
    transform = ccrs.PlateCarree(),
    #norm=colors.LogNorm(vmin=np.nanmin(TEP_0_interp),vmax=np.nanmax(TEP_0_interp)),
    cmap='viridis')
mygrid(ax1,grid=True)
ax1.set_extent(plot_extent, crs=ccrs.PlateCarree())
ax1.set_title('July',fontsize=20)
ax1.add_geometries([ring1], ccrs.PlateCarree(), facecolor='None', alpha=1, edgecolor='red',linewidth = 3)

ax2 = plt.subplot(1,2,2,projection = ccrs.NorthPolarStereo())
m2 = ax2.pcolormesh(londic05, latdic05, TEP_monthly_0_interp_SEP, 
    transform = ccrs.PlateCarree(),
    #norm=colors.LogNorm(vmin=np.nanmin(TEP_0_interp),vmax=np.nanmax(TEP_0_interp)),
    cmap='viridis')
mygrid(ax2,grid=True)
ax2.set_extent(plot_extent, crs=ccrs.PlateCarree())
ax2.set_title('September',fontsize=20)
ax2.add_geometries([ring1], ccrs.PlateCarree(), facecolor='None', alpha=1, edgecolor='red',linewidth = 3)


# colorbar
cbar = fig.colorbar(m1, ax=[ax1,ax2],orientation = 'horizontal',
                    fraction=0.1, pad=0.05, shrink = 0.8) 
cbar.set_label('[$\mu$g C L$^{{-1}}$]\nMonthly mean TEP conc. at 0-5 m ({0}-{1})'.format(
    years_last10[0],years_last10[-1]), 
               fontsize=20)
cbar.ax.tick_params(labelsize=20)

plt.tight_layout()

if savefig:
    plt.savefig(savepath+'TEP_interp_NPS_JUL_SEP_{0}_{1}.png'.format(
    years_last10[0],years_last10[-1]), dpi=300)

### Fram Strait Transect

In [ ]:
#box_FRAM =[-20, 12, 77, 82] ## see above

# old, S-N-Transect
#lon_start = 0
#lon_end = 5
#lat_start = 75
#lat_end = 83

# von Jackowski et al. (2020), figure 2
lon_start = -20
lon_end = 12
lat_start = 79
lat_end = 79
npoints   = 200
lonlat = pf.transect_get_lonlat(lon_start, lat_start, lon_end, lat_end, npoints)

if True:
    pf.plot_transect_map(lonlat, mesh)

print('Plotting transect at lat {0} to {1}, lon {2} to {3}'.format(lat_start,lat_end,lon_start,lon_end))
#pf.plot_transect_map(lonlat, mesh)

#### TEP

In [ ]:
#### July 

conv = 12.01 # [mmol C/m3] -> [mug /L]  
TEPall = TEP_seasonal_alldepths[6,:,:] * conv
time_label = 'July mean over {0}-{1}'.format(last_year-10,last_year)

# control: --------------------------------------------------
dist_label= 'Fram Strait \nTransect at lat {0}-{1}, lon {2}-{3}'.format(lat_start,lat_end,lon_start,lon_end)
print(dist_label)
#pf.plot_transect_map(lonlat, mesh)

# extract transect data
dist, tfTEP_Jul = pf.get_transect(TEPall,mesh,lonlat)

# set cbar levels --------------------------------------------------
lev_TEP = np.linspace(0,200,101)

### plot July --------------------------------------------------
#pf.plot_xyz?
pf.plot_xyz(mesh, 
            tfTEP_Jul,
            dist,
            maxdepth=100,
            xlabel='Fram Strait Transect [km]\nat {0}°N, {1}°W - {2}°E\n{3}'.format(
                lat_start,-lon_start,lon_end,time_label),
            title='TEP',
            label='TEP-C [$\mu$g/L]',
            levels=lev_TEP,
            figsize=(15,10),
            cmap='viridis')

In [ ]:
### September 

conv = 12.01 # [mmol C/m3] -> [mug /L]  
TEPall = TEP_seasonal_alldepths[8,:,:] * conv
time_label = 'September mean over {0}-{1}'.format(last_year-10,last_year)

# control: --------------------------------------------------
dist_label= 'Fram Strait \nTransect at lat {0}-{1}, lon {2}-{3}'.format(lat_start,lat_end,lon_start,lon_end)
print(dist_label)

# extract transect data
dist, tfTEP_Sep = pf.get_transect(TEPall,mesh,lonlat)

### plot September --------------------------------------------------
#pf.plot_xyz?
pf.plot_xyz(mesh, 
            tfTEP_Sep,
            dist,
            maxdepth=100,
            xlabel='Fram Strait Transect [km]\nat {0}°N, {1}°W - {2}°E\n{3}'.format(
                lat_start,-lon_start,lon_end,time_label),
            title='TEP',
            label='TEP-C [$\mu$g/L]',
            levels=lev_TEP,
            figsize=(15,10),
            cmap='viridis')

#### PCHO

In [ ]:
### Fram Strait Transect: PCHO

### July mean 
PCHO = PCHO_seasonal_alldepths[6,:,:]

# extract transect data
dist, tfPCHO_Jul = pf.get_transect(PCHO,mesh,lonlat)

# -------------------------------------------------------------------------------
### September mean 
PCHO = PCHO_seasonal_alldepths[8,:,:]

# extract transect data
dist, tfPCHO_Sep = pf.get_transect(PCHO,mesh,lonlat)

# -------------------------------------------------------------------------------
# control: 
dist_label= 'Fram Strait Transect at lat {0}-{1}, lon {2}-{3}'.format(lat_start,lat_end,lon_start,lon_end)
print(dist_label)

# set cbar levels 
lev_PCHO = np.linspace(0,7,71)

#### Chlorophyll

In [ ]:
### Fram Strait Transect: Chlorophyll

#### July mean 
Chl_SUM_Fram = Chl_seasonal_alldepths[6,:,:]

# extract transect data
dist, tfChl_Jul = pf.get_transect(Chl_SUM_Fram,mesh,lonlat)

# -------------------------------------------------------------------------------
#### September mean 
Chl_SUM_Fram = Chl_seasonal_alldepths[8,:,:]

# extract transect data
dist, tfChl_Sep = pf.get_transect(Chl_SUM_Fram,mesh,lonlat)

# -------------------------------------------------------------------------------
# control: 
dist_label= 'Fram Strait Transect at lat {0}-{1}, lon {2}-{3}'.format(lat_start,lat_end,lon_start,lon_end)
print(dist_label)

# set cbar levels 
lev_Chl = np.linspace(0,5,51)

#### Plotting Chl, PCHO, TEP

In [ ]:
### plotting July for Chl, PCHO, TEP

time_label = '\nJuly mean over {0}-{1}'.format(last_year-10,last_year)

pf.plot_xyz(mesh, 
            [tfChl_Jul,tfPCHO_Jul,tfTEP_Jul],
            dist,
            maxdepth=100,
            ncols=3,
            xlabel='Distance [km]',
            title=['Chl.a',
                   'Fram Strait Transect at {0}°N, {1}°W - {2}°E \nPCHO'.format(
                        lat_start,-lon_start,lon_end),
                   'TEP'],
            label=['Chl.a [mg/m3]','PCHO-C [mmol/m3]'+time_label,'TEP-C [mmol/m3]'],
            levels=[lev_Chl,lev_PCHO,lev_TEP],
            figsize=(15,6),
            cmap='viridis')


if savefig:
    plt.savefig(savepath+'TEP_Fram_Transect_July_{0}_{1}.png'.format(
    years_last10[0],years_last10[-1]), dpi=300)

In [ ]:
### plotting September for Chl, PCHO, TEP

time_label = '\nSeptember mean over {0}-{1}'.format(last_year-10,last_year)

pf.plot_xyz(mesh, 
            [tfChl_Sep,tfPCHO_Sep,tfTEP_Sep],
            dist,
            maxdepth=100,
            ncols=3,
            xlabel='Distance [km]',
            title=['Chl.a',
                   'Fram Strait Transect at {0}°N, {1}°W - {2}°E \nPCHO'.format(
                        lat_start,-lon_start,lon_end),
                   'TEP'],
            label=['Chl.a [mg/m3]','PCHO-C [mmol/m3]'+time_label,'TEP-C [mmol/m3]'],
            levels=[lev_Chl, lev_PCHO, lev_TEP],
            figsize=(15,6),
            cmap='viridis')


if savefig:
    plt.savefig(savepath+'TEP_Fram_Transect_September_{0}_{1}.png'.format(
    years_last10[0],years_last10[-1]), dpi=300)

### Seasonal Cycle

In [ ]:
TEP_seasonal_FRAM = TEP_seasonal_alldepths[:,mask_FRAM,:]
TEP_seasonal_AO = TEP_seasonal_alldepths[:,mask_AO,:]
TEP_seasonal_SO = TEP_seasonal_alldepths[:,mask_SO,:]

print(np.shape(TEP_seasonal_alldepths))
print(np.shape(TEP_seasonal_FRAM))

In [ ]:
#### now process data
verbose = True
num_levels = 11

### define storage containers for mean data in shape (months, depths)
TEP_glob_m = np.zeros((len(months),num_levels))
TEP_FRAM_m = np.zeros((len(months),num_levels))
TEP_ao_m = np.zeros((len(months),num_levels))
TEP_so_m = np.zeros((len(months),num_levels))

for idepth in range(num_levels): #len(mesh.zlev)-1):
    if verbose:
        print('Processing level depth = {} m'.format(mesh.zlev[idepth]))
    
    #print(np.shape(TEP_seasonal_alldepths[:,:,idepth]))
    #print(np.shape(TEP_seasonal_AO[:,:,idepth]))
    
    TEP_glob_m[:,idepth] = np.nanmean(TEP_seasonal_alldepths[:,:,idepth],axis=1)
    TEP_FRAM_m[:,idepth] = np.nanmean(TEP_seasonal_FRAM[:,:,idepth],axis=1)
    TEP_ao_m[:,idepth] = np.nanmean(TEP_seasonal_AO[:,:,idepth],axis=1)
    TEP_so_m[:,idepth] = np.nanmean(TEP_seasonal_SO[:,:,idepth],axis=1)

In [ ]:
#### plotting


fig = plt.figure(figsize=(10,7), facecolor='w', edgecolor='k', tight_layout = True)
ylim = (0,15.5)

plt.subplot(2, 2, 1)
plt.plot(months,TEP_glob_m[:,0],'.-',label='0-5 m')
plt.plot(months,TEP_glob_m[:,1],'.-',label='5-10 m')
plt.plot(months,TEP_glob_m[:,2],'.-',label='10-20 m')
plt.plot(months,TEP_glob_m[:,3],'.-',label='20-30 m')
plt.title('Global mean TEP ({0}-{1})'.format(years_last10[0],years_last10[-1]))
plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')

plt.subplot(2, 2, 2)
plt.plot(months,TEP_ao_m[:,0],'.-',label='0-5 m')
plt.plot(months,TEP_ao_m[:,1],'.-',label='5-10 m')
plt.plot(months,TEP_ao_m[:,2],'.-',label='10-20 m')
plt.plot(months,TEP_ao_m[:,3],'.-',label='20-30 m')
plt.title('North of 70 N: mean monthly TEP')
#plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')

plt.subplot(2, 2, 3)
plt.plot(months,TEP_so_m[:,0],'.-',label='0-5 m')
plt.plot(months,TEP_so_m[:,1],'o',label='5-10 m')
plt.plot(months,TEP_so_m[:,2],'*',label='10-20 m')
plt.plot(months,TEP_so_m[:,3],'*',label='20-30 m')
plt.title('S of -50 mean TEP')
plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')

plt.subplot(2, 2, 4)
plt.plot(months,TEP_FRAM_m[:,0],'.-',label='0-5 m')
plt.plot(months,TEP_FRAM_m[:,1],'.-',label='5-10 m')
plt.plot(months,TEP_FRAM_m[:,2],'.-',label='10-20 m')
plt.plot(months,TEP_FRAM_m[:,3],'.-',label='20-30 m')
plt.title('Fram Strait mean TEP')
#plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')


if savefig:
    plt.savefig(savepath+'TEP_monthly_multipledepths_{0}_{1}.png'.format(
    years_last10[0],years_last10[-1]),dpi=300)

In [ ]:
### volume-weighted mean
# -> TEP_monthly.data_monthly_wm[:,mask_FRAM]

plt.plot(np.nanmean(TEP_monthly.data_monthly_wm[:,mask_FRAM],axis=1))
plt.plot(np.std(TEP_monthly.data_monthly_wm[:,mask_FRAM],axis=1))



In [ ]:
### plotting Arctic TEP [mug /L]    

conv = 12.01 # [mmol C/m3] -> [mug /L]  

fig = plt.figure(figsize=(5,4), facecolor='w', edgecolor='k', tight_layout = True)
ylim = (-10,190)
CB_cycle = ['#f28e2b','#e15759','#b07aa1','#59a14f','#4e79a7','#76b7b2','#9c755f','#ff9da7']

# -------------------------------------------------------------------------------
# adding model data
plt.plot(months,TEP_FRAM_m[:,0]*conv,'.-',color=CB_cycle[3],label='0-5')
plt.plot(months,TEP_FRAM_m[:,1]*conv,'.-',color=CB_cycle[4],label='5-10')
plt.plot(months,TEP_FRAM_m[:,2]*conv,'.-',color=CB_cycle[2],label='10-20')
#plt.plot(months,TEP_FRAM_m[:,3]*conv,'.-',color=CB_cycle[2],label='20-30 m')
#plt.plot(months,TEP_FRAM_m[:,10]*conv,'.-',color=CB_cycle[2],label='90-100 m')
    
# -------------------------------------------------------------------------------
# adding weighted mean
plt.plot(months, np.nanmean(TEP_monthly.data_monthly_wm[:,mask_FRAM],axis=1)*conv,
        linestyle='-',marker='.',color='dimgrey',
        label='0-100')
plt.errorbar(months, np.nanmean(TEP_monthly.data_monthly_wm[:,mask_FRAM],axis=1)*conv,
             yerr = np.std(TEP_monthly.data_monthly_wm[:,mask_FRAM],axis=1)*conv,
             capsize=4, linestyle='', color='dimgrey')

# -------------------------------------------------------------------------------
# adding observation data
plt.axvline(x=6, linewidth=.70, linestyle='--', color='k')
plt.axvline(x=7, linewidth=.70, linestyle='--', color='k')
plt.axvline(x=9, linewidth=.70, linestyle='--', color='k')

# Engel 2020
plt.plot(6,8.4, linestyle='', marker='s', color=CB_cycle[1],label='0-200 [E]')
plt.errorbar(6,8.4,yerr=6.1, 
             capsize=4,linestyle='',color=CB_cycle[1])

# v Jackowski 2020
plt.plot((7,9),(21.4,7.1),linestyle='', marker='D', color=CB_cycle[0], label='0-100 [vJ]')
plt.errorbar((7,9),(21.4,7.1),yerr=(14.5,5.2),
             capsize=4,linestyle='',color=CB_cycle[0])

# -------------------------------------------------------------------------------
# figure outline
plt.axhline(y=0, linewidth=.70, linestyle='-', color='k')
plt.xlabel('Month\n\nFram Strait, {0}-{1}'.format(
    years_last10[0],years_last10[-1]),fontsize=13)
plt.ylabel(r'TEP [$\mu$g C L$^{-1}$]',fontsize=13)
plt.ylim(ylim)

# -------------------------------------------------------------------------------
# Legend
plt.legend(loc='upper left',fontsize=13,title='Depth range [m]')
#[m1,m2,m3,m4,m5,o1,o2],['0-5 m','5-10 m','10-20 m','20-30 m','0-100 m','E: 0-200 m','vJ: 0-100 m'],

if savefig:
    plt.savefig(savepath+'TEP_monthly_multipledepths_FRAM_{0}_{1}.png'.format(
    years_last10[0],years_last10[-1]),dpi=300)

In [ ]:
### differences in Southern Ocean

fig = plt.figure(figsize=(12,8), facecolor='w', edgecolor='k', tight_layout = True)
ylim = (-0.01,0.01)

plt.subplot(2, 3, 3)
plt.plot(months,(TEP_so_m[:,0]-TEP_so_m[:,0]),'.-',label='0 m')
plt.plot(months,TEP_so_m[:,1]-TEP_so_m[:,0],'o-',label='5 m')
plt.plot(months,TEP_so_m[:,2]-TEP_so_m[:,0],'*-',label='10 m')
plt.plot(months,(TEP_so_m[:,3]-TEP_so_m[:,0])/50,'*-',label='0.02*(20 m)')
plt.title('S of -50 mean TEP')
plt.xlabel('months\nSurface layer - layer at certain depth')
plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')

### Monthly Chl.a : TEP ratio

TEP-C [$\mu$mol L$^{-1}$] = 0.05 TEP$_{color}$ [mg Xeq. L$^{-1}$]

figure y= log TEP XG eq , x= log Chla cmop. Zamanillo 2019 and Yamada 2015

In [ ]:
#### convert and cut to Fram Strait 

TEP_FRAM_0_JUL_mugXeqL = (TEP_seasonal_alldepths[6,mask_FRAM,0] *20 *1000).ravel()
TEP_FRAM_0_SEP_mugXeqL = (TEP_seasonal_alldepths[8,mask_FRAM,0] *20 *1000).ravel()

Chl_FRAM_0_JUL = Chl_seasonal_alldepths[6,mask_FRAM,0].ravel()
Chl_FRAM_0_SEP = Chl_seasonal_alldepths[8,mask_FRAM,0].ravel()

In [ ]:
### set Chl.a treshold and plotting relation

x1, y1 = np.log10(set_Chl_min(0.01,Chl_FRAM_0_JUL,TEP_FRAM_0_JUL_mugXeqL))

x2, y2 = np.log10(set_Chl_min(0.01,Chl_FRAM_0_SEP,TEP_FRAM_0_SEP_mugXeqL))

In [ ]:
### plotting 

plt.figure(figsize=(10,10))

plt.subplot(2,2,1)
plt.plot(x1,y1,'.')
plt.ylim(0,6)
plt.xlim(-2.5,2.5)
plt.xlabel('log10 Chl.a [mg m$^{-3}$]\nmin(Chl.a) = 0.01 mg m$^{-3}$\nFram Strait, July')
plt.ylabel('log10 TEP [$\mu$g Xeq. L$^{-1}$]')
res, log = get_res(x1,y1)
plt.plot(x1, res.intercept + res.slope*x1, 'r', label='fitted line')
plt.text(-2,0.5,log)

plt.subplot(2,2,2)
plt.plot(x2,y2,'.')
plt.ylim(0,6)
plt.xlim(-2.5,2.5)
plt.xlabel('log10 Chl.a [mg m$^{-3}$]\nmin(Chl.a) = 0.1 mg m$^{-3}$\nFram Strait')
#plt.ylabel('log10 TEP [$\mu$g Xeq. L$^{-1}$]')
res, log = get_res(x2,y2)
plt.plot(x2, res.intercept + res.slope*x2, 'r', label='fitted line')
plt.text(-2,0.5,log)


plt.tight_layout()

if False:
    plt.savefig(savepath+'Chl_TEP_FRAM_JUL_SEP_regression.png', dpi=300)

### Atlantic Transect

In [ ]:
### TEP Atlantic Transect

lon_start = -60
lat_start = -80
lon_end = 10
lat_end = 83
npoints   = 100
lonlat = pf.transect_get_lonlat(lon_start, lat_start, lon_end, lat_end, npoints)

if True:
    pf.plot_transect_map(lonlat, mesh)

print('Plotting transect at lat {0} to {1}, lon {2} to {3}'.format(lat_start,lat_end,lon_start,lon_end))

In [ ]:
### get data
TEPall = pf.get_data(resultpath, "TEP", years_last10, mesh=mesh, runid=runid, how="mean")

### extract transect data
dist, tfTEP = pf.get_transect(TEPall,mesh,lonlat)

### plotting
pf.plot_xyz(mesh, 
            tfTEP,
            dist,
            maxdepth=600,
            ncols=1,
            xlabel='Distance',
            label='TEP-C [mmol/m3]',
            #levels=np.linspace(-2,2, 41),
            cmap='viridis')

### Global and regional mean

In [ ]:
### calculating mean TEP per year 

TEP00glob_total = np.zeros(len(years))
TEP05glob_total  = np.zeros(len(years))
TEP10glob_total = np.zeros(len(years))
TEP20glob_total   = np.zeros(len(years))

TEP00ao_total = np.zeros(len(years))
TEP05ao_total = np.zeros(len(years))
TEP10ao_total = np.zeros(len(years))
TEP20ao_total = np.zeros(len(years))

TEP00FRAM_total = np.zeros(len(years))
TEP05FRAM_total = np.zeros(len(years))
TEP10FRAM_total = np.zeros(len(years))
TEP20FRAM_total = np.zeros(len(years))

TEP00so_total = np.zeros(len(years))
TEP05so_total = np.zeros(len(years))
TEP10so_total = np.zeros(len(years))
TEP20so_total = np.zeros(len(years))


for i in range(len(years)):
    ind_year = int(years[i])
    #print('loading {0}'.format(ind_year))
    # TEP:units = "mmolC/m3"
    
    # 0m ############################
    TEP00glob = pf.get_data(resultpath, "TEP", ind_year, mesh, how="mean",
                            depth= 0,
                            compute=True, runid='fesom', silent=True)
    
    TEP00glob_total[i] = np.nanmean(TEP00glob)#*nod_area_surface)
    TEP00ao_total[i] = np.nanmean(TEP00glob[mask_AO])
    TEP00FRAM_total[i] = np.nanmean(TEP00glob[mask_FRAM])
    TEP00so_total[i] = np.nanmean(TEP00glob[mask_SO])
    
    # 5 m ############################
    TEP05glob = pf.get_data(resultpath, "TEP", ind_year, mesh, how="mean", 
                        depth= 5,
                        compute=True, runid='fesom', silent=True)

    TEP05glob_total[i] = np.nanmean(TEP05glob)
    TEP05ao_total[i] = np.nanmean(TEP05glob[mask_AO])
    TEP05FRAM_total[i] = np.nanmean(TEP05glob[mask_FRAM])
    TEP05so_total[i] = np.nanmean(TEP05glob[mask_SO])

    # 10 m ############################
    TEP10glob = pf.get_data(resultpath, "TEP", ind_year, mesh, how="mean", 
                        depth= 10,
                        compute=True, runid='fesom', silent=True)

    TEP10glob_total[i] = np.nanmean(TEP10glob)
    TEP10ao_total[i] = np.nanmean(TEP10glob[mask_AO])
    TEP10FRAM_total[i] = np.nanmean(TEP10glob[mask_FRAM])
    TEP10so_total[i] = np.nanmean(TEP10glob[mask_SO])
    
    # 20 m ############################
    TEP20glob = pf.get_data(resultpath, "TEP", ind_year, mesh, how="mean", 
                        depth= 20,
                        compute=True, runid='fesom', silent=True)

    TEP20glob_total[i] = np.nanmean(TEP20glob)
    TEP20ao_total[i] = np.nanmean(TEP20glob[mask_AO])
    TEP20FRAM_total[i] = np.nanmean(TEP20glob[mask_FRAM])
    TEP20so_total[i] = np.nanmean(TEP20glob[mask_SO])

In [ ]:
### plotting total TEP         

fig = plt.figure(figsize=(12,8), facecolor='w', edgecolor='k', tight_layout = True)
ylim = (0,7)

# 0 m ############################

plt.subplot(2, 2, 1)
plt.plot(years,TEP00glob_total,'.-',label='0-5 m')
plt.plot(years,TEP05glob_total,'.-',label='5-10 m')
plt.plot(years,TEP10glob_total,'.-',label='10-20 m')
plt.plot(years,TEP20glob_total,'.-',label='20-30 m')
plt.title('Global mean TEP')
plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')

plt.subplot(2, 2, 2)
plt.plot(years,TEP00ao_total,'.-',label='0-5 m')
plt.plot(years,TEP05ao_total,'.-',label='5-10 m')
plt.plot(years,TEP10ao_total,'.-',label='10-20 m')
plt.plot(years,TEP20ao_total,'.-',label='20-30 m')
plt.title('N of 70N mean TEP')
plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')

plt.subplot(2, 2, 3)
plt.plot(years,TEP00so_total,'.-',label='0-5 m')
plt.plot(years,TEP05so_total,'.-',label='5-10 m')
plt.plot(years,TEP10so_total,'.-',label='10-20 m')
plt.plot(years,TEP20so_total,'.-',label='20-30 m')
plt.title('S. Ocean mean TEP')
plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')

plt.subplot(2, 2, 4)
plt.plot(years,TEP00FRAM_total,'.-',label='0-5 m')
plt.plot(years,TEP05FRAM_total,'.-',label='5-10 m')
plt.plot(years,TEP10FRAM_total,'.-',label='10-20 m')
plt.plot(years,TEP20FRAM_total,'.-',label='20-30 m')
plt.title('Fram Strait mean TEP')
plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')

if savefig:
    plt.savefig(savepath+'TEP_timeseries.png',dpi=300)

## PCHO

In [ ]:
### Atlantic transect 

lon_start = -60
lat_start = -80
lon_end = 10
lat_end = 83
npoints   = 100
lonlat = pf.transect_get_lonlat(lon_start, lat_start, lon_end, lat_end, npoints)

### load data
PCHOall = pf.get_data(resultpath, "PCHO", years=years_last10, mesh=mesh, runid=runid, how="mean")#, depth=depth)

### extract transect data
dist, tfPCHO = pf.get_transect(PCHOall,mesh,lonlat)

### plotting
pf.plot_xyz(mesh, 
            tfPCHO,
            dist,
            maxdepth=600,
            ncols=1,
            xlabel='Distance',
            label='PCHO [mmol/m3]',
            #levels=np.linspace(-2,2, 41),
            cmap='viridis',
            figsize=(15, 6))

## Chlorophyll a

In [ ]:
#### load only last 10 years, 1 uppermost depth layer1

Chl = Chlsurf_OCCCI_comp(runid,resultpath,savepath,meshpath,matfileChlsurf,
                          last_year-10,last_year,
                         n_levels=1,
                          mapproj=mapproj,savefig=savefig)

In [ ]:
#### load only last 10 years, 2 uppermost depth layers

Chl_n2 = Chlsurf_OCCCI_comp(runid,resultpath,savepath,meshpath,matfileChlsurf,
                          last_year-10,last_year,
                         n_levels=2,
                          mapproj=mapproj,savefig=savefig)

# --- EXTRA ---

# Intracellular Carbon

In [ ]:
# Diatom carbon
depth = 0
NCfesom = resultpath + '/DiaC.fesom.1948.nc'
#!ncdump -h $NCfesom

label='Diatom intracell. Carbon {0}-{1} at {2} m'.format(last_year-10,last_year,depth)
units = 'intracellular C [mmol C m$^{-3}$]'

DiaC = pf.get_data(resultpath, "DiaC", years=years_last10,
                   mesh=mesh, runid=runid, how="mean", depth=depth)
fig = pf.plot(mesh, DiaC, mapproj = mapproj, units=units, titles=label)

In [ ]:
# Phytplankton carbon

NCfesom = resultpath + '/PhyC.fesom.1948.nc'
#!ncdump -h $NCfesom

label='Phytoplankton intracell. Carbon {0}-{1} at {2} m'.format(last_year-10,last_year,depth)
units = 'intracellular C [mmol C m$^{-3}$]'

PhyC = pf.get_data(resultpath, "PhyC", years=years_last10,
                   mesh=mesh, runid=runid, how="mean", depth = depth)

fig = pf.plot(mesh, PhyC, mapproj = mapproj, units=units, titles=label)

# Detritus Carbon

In [ ]:
NCfesom = resultpath + '/DetC.fesom.1948.nc'
#!ncdump -h $NCfesom

depth = 0

label='DetC {0}-{1} at {2} m'.format(last_year-10,last_year,depth)
units = 'Detritus C [mmol m$^{-3}$]'

DetC = pf.get_data(resultpath, "DetC", years_last10, mesh, runid=runid, how="mean", compute=True, depth=depth)
fig = pf.plot(mesh, DetC, mapproj = mapproj, units=units, titles=label)

# extracellular carbon

In [ ]:
NCfesom = resultpath + '/EOC.fesom.1948.nc'
!ncdump -h $NCfesom

#label='TEP {0}-{1} at {2} m'.format(years_last10[0],years_last10[-1],depth)
#units = 'TEP-C [mmol m$^{-3}$]'
#TEP = pf.get_data(resultpath, "EOC", years_last10, mesh, runid=runid, how="mean", compute=True, depth=depth)
#fig = pf.plot(mesh, TEP, mapproj = mapproj, units=units, titles=label)

# NPP

In [ ]:
### map of total Diatom NPP 

NCfesom = resultpath + '/NPPd.fesom.1948.nc'
#!ncdump -h $NCfesom

# 12.01* data --> mmolC/m2/d to mg C/m2/d

label='Diatom NPP {0}-{1}'.format(last_year-10,last_year)
units = 'Mean NPP [mg C m$^{-2}$ d$^{-1}$]'

NPPd = pf.get_data(resultpath, "NPPd", 
                   years_last10,
                   mesh, runid=runid, how="mean", compute=True, silent=True)

#print(np.shape(NPPd))
# NPP is already vertically integrated!

NPPd = NPPd * 12.01
fig = pf.plot(mesh, NPPd, mapproj = mapproj, units=units, titles=label, 
              levels = (0,1100,56),
              cmap='viridis')

In [ ]:
### map of nanophytoplankton NPP

NCfesom = resultpath + '/NPPn.fesom.1948.nc'
#!ncdump -h $NCfesom

# 12.01* data --> mg C/m2/d

label='Nanophytoplankton NPP {0}-{1}'.format(last_year-10,last_year)
units = 'Mean NPP [mg C m$^{-2}$ d$^{-1}$]'

NPPn = pf.get_data(resultpath, "NPPn", 
                   years_last10,
                   mesh, runid=runid, how="mean", compute=True,silent=True)

#print(np.shape(NPPn))
# NPP is already vertically integrated!

NPPn = NPPn *12.01

fig = pf.plot(mesh, NPPn, mapproj = mapproj, units=units, titles=label, 
              levels = (0,1100,56),
              cmap='viridis')

In [ ]:
### total NPP

label='Total Net Primary Production {0}-{1}'.format(last_year-10,last_year)
units = 'Mean total NPP [mg C m$^{-2}$ d$^{-1}$]'

NPPtot = NPPd + NPPn
fig = pf.plot(mesh, NPPtot, mapproj = mapproj, units=units, titles=label, 
              levels = (0,1100,56),
              cmap='viridis')

# DIC

In [ ]:
### DIC

depth = 0
NCfesom = resultpath + '/DIC.fesom.1948.nc'
#!ncdump -h $NCfesom

label='Dissolved inorganic Carbon {0}-{1} at {2} m'.format(last_year-10,last_year,depth)
units = 'DIC [mmol  m$^{-3}$]'
DIC = pf.get_data(resultpath, "DIC", years_last10,mesh, how="mean", compute=True, depth=depth)
fig = pf.plot(mesh, DIC, mapproj = mapproj, units=units, titles=label)


# Monthly sea ice concentration

In [ ]:
aIce_monthly = get_monthly_data('a_ice', years=years_last10, verbose=True, only2D = True)


In [ ]:
### interpolation

# July
aIce_monthly_interp_JUL = pf.fesom2regular(data = aIce_monthly[6,:],mesh = mesh,lons = londic,lats = latdic)

# September
aIce_monthly_interp_SEP = pf.fesom2regular(data = aIce_monthly[8,:],mesh = mesh,lons = londic,lats = latdic)


In [ ]:
units = 'Ice concentration [%] \nMean of {0}-{1}'.format(years_last10[0],years_last10[-1])
cmap = 'BuPu'

# North Polar Stereo Projection
fig = plt.figure(figsize=(15,10))
plot_extent = [-180, 180, 70, 90]

# July
ax1 = plt.subplot(1,2,1,projection = ccrs.NorthPolarStereo())
m1 = ax1.pcolormesh(londic, latdic, aIce_monthly_interp_JUL, 
    transform = ccrs.PlateCarree(),
    cmap=cmap)

mygrid(ax1,grid=True)

ax1.set_extent(plot_extent, crs=ccrs.PlateCarree())
ax1.set_title('July',fontsize=20)
ax1.add_geometries([ring1], ccrs.PlateCarree(), facecolor='None', alpha=1, edgecolor='red',linewidth = 3)

# September
ax2 = plt.subplot(1,2,2,projection = ccrs.NorthPolarStereo())
m2 = ax2.pcolormesh(londic, latdic, aIce_monthly_interp_SEP, 
    transform = ccrs.PlateCarree(),
    cmap=cmap)

mygrid(ax2,grid=True)

ax2.set_extent(plot_extent, crs=ccrs.PlateCarree())
ax2.set_title('September',fontsize=20)
ax2.add_geometries([ring1], ccrs.PlateCarree(), facecolor='None', alpha=1, edgecolor='red',linewidth = 3)


# colorbar
cbar = fig.colorbar(m1, ax=[ax1,ax2],orientation = 'horizontal',
                    fraction=0.1, pad=0.05, shrink = 0.8) 
cbar.set_label(units, fontsize=20)
cbar.ax.tick_params(labelsize=20)

plt.tight_layout()

if savefig:
    plt.savefig(savepath+'aIce_interp_NPS_JUL_SEP_{0}_{1}.png'.format(
    years_last10[0],years_last10[-1]), dpi=300)

# Export to html

In [ ]:
stop

In [ ]:
!jupyter nbconvert TEP_detailed_analysis.ipynb --output-dir=$htmlpath --output $htmlname --to html --TemplateExporter.exclude_input=True


# Backup

In [ ]:
############# BACKUP ################
### calculating mean TEP per month 

# specifications
only_FRAM = False
verbose = True

num_levels = 11
depth_list = (0,5,10,20,30,40,50,60,70,80,90)

### Control: 
### Level depth, layer depth, layer thickness
print(mesh.zlev[:11], diag['Z'].values[:11], delta_z[:11])

# define storage containers for mean data in shape (months, depths)
TEP_glob_m = np.zeros((len(months),len(depth_list)))
TEP_FRAM_m = np.zeros((len(months),len(depth_list)))

TEP_glob_aux = np.zeros((len(years_last10),len(months)))
TEP_FRAM_aux = np.zeros((len(years_last10),len(months)))


TEP_ao_m = np.zeros((len(months),len(depth_list)))
TEP_ao_aux = np.zeros((len(years_last10),len(months)))

TEP_so_m = np.zeros((len(months),len(depth_list)))
TEP_so_aux = np.zeros((len(years_last10),len(months)))

#### now load and process data

for idepth in range(num_levels):
    if verbose:
        print('Processing level depth = {} m'.format(mesh.zlev[idepth]))
    
    # load data for one depth and calculate monthly mean
    for i in range(len(years_last10)):
        ind_year = int(years_last10[i])
        
        if verbose:
            print('loading {0}'.format(ind_year))

        TEP_raw = pf.get_data(resultpath, "TEP", ind_year, mesh, how="ori", 
                            depth=int(mesh.zlev[idepth]), compute=False,  silent=True)
        # TEP:units = "mmolC/m3"
        
        TEP_glob_aux[i,:] = np.nanmean(TEP_raw,axis=1)
        TEP_FRAM_aux[i,:] = np.nanmean(TEP_raw[:,mask_FRAM],axis=1)
        
        if not only_FRAM:
            TEP_ao_aux[i,:] = np.nanmean(TEP_raw[:,mask_AO],axis=1)
            TEP_so_aux[i,:] = np.nanmean(TEP_raw[:,mask_SO],axis=1)
        
    #TEP_mmean = np.nanmean(TEP_glob_aux,axis=0)
    TEP_glob_m[:,idepth] = np.nanmean(TEP_glob_aux,axis=0)
    TEP_FRAM_m[:,idepth] = np.nanmean(TEP_FRAM_aux,axis=0)
    
    if not only_FRAM:
        TEP_ao_m[:,idepth] = np.nanmean(TEP_ao_aux,axis=0)
        TEP_so_m[:,idepth] = np.nanmean(TEP_so_aux,axis=0)
        
print(np.shape(TEP_FRAM_m)) # months, depths


